In [2]:
import pandas as pd
import os
import re
import emoji
import nltk

nltk.download('words')

[nltk_data] Downloading package words to /Users/vinay/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [167]:
path = 'task_3/toi/'

In [168]:
def clean(row):
    text = row['text']
    text = ''.join(text.split('\n')[:-1])
    text = ''.join(text.split(':')[1:])
    return text.strip()

lis = []
for folder, _, files in os.walk(path):
    for file in files:
        if '.csv' in file:
            x = pd.read_csv(f'{folder}{file}')
            x['date'] = file.split('.')[0]
            lis.append(x)
            
df = pd.concat(lis)
df = df.drop(labels='Unnamed: 0', axis=1)
df.date = pd.to_datetime(df.date)
df.text = df.apply(clean, 1)
df.location = df.apply(lambda x: x['location'].lower(), 1)

df.head()

,title,text,location,date
0,2 more riot accused arrested,the city police arrested two simi activists on...,kanpur,2018-09-19
1,Rajnath drops another minister,chief minister rajnath singh on tuesday droppe...,lucknow,2018-09-19
2,"Chauhan rally, a damp squib",former khalistan ideologue jagjit singh chauha...,tanda (hoshiarpur),2018-09-19
3,Paswan rules out divestment in coal sector,union minister of coal and mines ram vilas pas...,dhanbad,2018-09-19
4,No change in upper age limit for govt jobs,the state cabinet on monday decided that the u...,patna,2018-09-19


In [45]:
df.to_csv('toi.csv')

In [76]:
import difflib
from difflib import SequenceMatcher

In [83]:
t1 = pd.read_csv('task1_valid.csv')
t1 = t1.where(pd.notnull(t1), None)
t1.columns

Index(['Unnamed: 0', 'notes', 'event_date', 'source', 'fatalities',
       'event_type', 'sub_event_type', 'actor1', 'inter1', 'actor2', 'inter2',
       'interaction', 'location', 'pred_location', 'pred_fatalities'],
      dtype='object')

In [61]:
locs = t1['location']
pred_locs = t1[t1['pred_location'].notna()].pred_location.values

In [69]:
res = {}
for loc in set(locs):
    res[loc] = difflib.get_close_matches(loc, set(pred_locs), n=5, cutoff=0.7)
        
    # res[loc] = temp.sort(key=lambda x: x[1])

In [70]:
list(res.keys())[:4]

['Lamu', 'Bulo Burto', 'Malanje', 'Harare']

In [75]:
res['Harare']

['Harare', 'Hararyale', 'Haradhere', 'Haraale', 'Hararyaale']

In [78]:
similar('Harare', 'Hararyale')

0.8

In [79]:
similar('Harare', 'Haraale')

0.7692307692307693

In [131]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def get_closest_matches(df, actual_col, pred_col, thresh):
    actual = df[df[actual_col].notna()][actual_col].values
    preds = df[df[pred_col].notna()][pred_col].values
    
    res = {}
    for x in set(actual):
        tmp = []
        for y in set(preds):
            score = similar(x, y)
            if score >= thresh:
                tmp.append((y, score))
        tmp.sort(key=lambda x: x[1], reverse=True)
        res[x] = tmp
    return res

def replace_closest_matches(df, actual_col, pred_col, col_map, n):
    actual = df[df[actual_col].notna()][actual_col].values
    preds = df[df[pred_col].notna()][pred_col].values
    mp = {}
    
    for a in actual:
        m = min(n, len(col_map.get(a, [])))
        for i in range(m):
            mp[col_map[a][i][0]] = a
            
    df[pred_col] = df[pred_col].apply(lambda x: mp[x] if x in mp else x)
    return mp, df

In [142]:
col_map = get_closest_matches(t1, 'location', 'pred_location', 0.7)

In [139]:
mp, new_df = replace_closest_matches(t1, 'location', 'pred_location', col_map, n=3)

In [140]:
new_df['location'].unique().shape

(1336,)

In [141]:
new_df['pred_location'].unique().shape

(2570,)

In [3]:
tw = pd.read_csv('tweets.csv')
tw.head()

,Datetime,Tweet Id,Text,Username
0,2020-11-05 23:34:24+00:00,1.324500e+18,"Pakistani Army Extracts From Its Own Farmers, ...",HindiNewsLates1
1,2020-11-05 22:09:16+00:00,1.324470e+18,{editor} Non-stun export ban sparks mass farme...,ukagri
2,2020-11-05 21:57:50+00:00,1.324470e+18,PTI itself kept Islamabad hijacked for months ...,Gulalai_Ismail
3,2020-11-05 21:54:22+00:00,1.324470e+18,Malik Ishfaq Langrial passes away after Punjab...,AanuJanu1
4,2020-11-05 21:30:24+00:00,1.324460e+18,"Farmers postpone protest after baton-charge, a...",RelationsState


In [4]:
words = set(nltk.corpus.words.words())

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet

tw['tweet'] = tw['Text'].map(lambda x: cleaner(x))

In [5]:
tw.head()

,Datetime,Tweet Id,Text,Username,tweet
0,2020-11-05 23:34:24+00:00,1.324500e+18,"Pakistani Army Extracts From Its Own Farmers, ...",HindiNewsLates1,"Army From Its Own , Protest , Then ANN"
1,2020-11-05 22:09:16+00:00,1.324470e+18,{editor} Non-stun export ban sparks mass farme...,ukagri,{ editor } Non - stun export ban sparks mass f...
2,2020-11-05 21:57:50+00:00,1.324470e+18,PTI itself kept Islamabad hijacked for months ...,Gulalai_Ismail,itself kept for in attempt to throw the democr...
3,2020-11-05 21:54:22+00:00,1.324470e+18,Malik Ishfaq Langrial passes away after Punjab...,AanuJanu1,Malik away after Police torture . He sustained...
4,2020-11-05 21:30:24+00:00,1.324460e+18,"Farmers postpone protest after baton-charge, a...",RelationsState,"postpone protest after baton - charge , –"


In [23]:
tw['date'] = pd.to_datetime(tw['Datetime']).dt.date
df = tw.drop(labels=['Datetime', 'Tweet Id', 'Username', 'Text'], axis=1)

In [24]:
df.head()

,tweet,date
0,"Army From Its Own , Protest , Then ANN",2020-11-05
1,{ editor } Non - stun export ban sparks mass f...,2020-11-05
2,itself kept for in attempt to throw the democr...,2020-11-05
3,Malik away after Police torture . He sustained...,2020-11-05
4,"postpone protest after baton - charge , –",2020-11-05


In [27]:
df = df.groupby(['date'], as_index=False).agg({'tweet': ' | '.join})

In [28]:
df.head()

,date,tweet
0,2020-11-05,"Army From Its Own , Protest , Then ANN | { edi..."
1,2020-11-06,: Northern current situation amid ’ protest | ...
2,2020-11-07,I ' m from the ... It ' s not all as great as ...
3,2020-11-08,After all this government is not listening to ...
4,2020-11-09,civil demand the arrest of police following th...


In [30]:
df.date.min()

datetime.date(2020, 11, 5)

In [31]:
df.date.max()

datetime.date(2021, 3, 4)

In [29]:
df.to_csv('tweets.csv')